In [27]:
import pandas as pd
import numpy as np
import os

from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
##connect to db
#engine = create_engine(os.getenv("PG_CONN"))
#df = pd.read_sql("SELECT * FROM jobs", engine)

In [13]:
#jobs = pd.read_sql("SELECT * FROM job", engine)
jobs = pd.read_csv("job.csv", index_col=0)
jobs.head()

,title,contract_type,contract_time,created,adref,redirect_url,salary_is_predicted,company_id,location_id,category_id
job_id,,,,,,,,,,
5485243050,Zerspanungsmechaniker (m/w/d),NaN,full_time,2025-11-07 13:05:18,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5485243050?utm_m...,0,1,1,1
5494004615,Senior Software Engineer FIND - Solr (Remote -...,NaN,full_time,2025-11-13 15:51:45,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ5NDAwNDYxNSIsI...,https://www.adzuna.de/details/5494004615?utm_m...,0,2,2,1
5489819814,(Senior) Softwareentwickler Java / Java EE (m/...,permanent,full_time,2025-11-10 21:16:57,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ4OTgxOTgxNCIsI...,https://www.adzuna.de/details/5489819814?utm_m...,0,3,3,1
5482662909,Produktionsmitarbeiter Feinmechanik (m/w/d),NaN,full_time,2025-11-05 20:35:43,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5482662909?utm_m...,0,4,4,1
5467966903,Lagermitarbeiter (m/w/d),NaN,full_time,2025-10-28 14:05:56,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5467966903?utm_m...,0,5,5,1


In [21]:
jobs.describe()

,salary_is_predicted,company_id,location_id,category_id
count,50.0,50.000000,50.000000,50.0
mean,0.0,18.620000,24.380000,1.0
std,0.0,16.498472,14.878954,0.0
min,0.0,1.000000,1.000000,1.0
25%,0.0,3.000000,11.250000,1.0
50%,0.0,11.500000,23.500000,1.0
75%,0.0,33.500000,36.750000,1.0
max,0.0,50.000000,50.000000,1.0


In [14]:
#com = pd.read_sql("SELECT * FROM company", engine)
com = pd.read_csv("company.csv", index_col=0)
com.head()

,company_name
company_id,
4,persona service AG & Co. KG • Niederlassung: S...
5,persona service AG & Co. KG • Niederlassung: P...
9,persona service AG & Co. KG • Niederlassung: K...
11,persona service AG & Co. KG • Niederlassung: B...
12,persona service AG & Co. KG • Niederlassung: F...


In [15]:
#cat = pd.read_sql("SELECT * FROM category", engine)
cat = pd.read_csv("category.csv", index_col=0)
cat.head()

,tag,label
category_id,,
1,it-jobs,IT-Stellen


In [16]:
#locs = pd.read_sql("SELECT * FROM location", engine)
locs = pd.read_csv("location.csv", index_col=0)
locs.head()

,display_name,latitude,longitude,country,city
location_id,,,,,
1,"Rosenfeld, Zollernalbkreis",48.28660,8.72455,Deutschland,Zollernalbkreis
3,"Almoshof, Nürnberg",49.49109,11.06150,Deutschland,Nürnberg
4,"Reichenberg, Würzburg (Kreis)",49.73188,9.91563,Deutschland,Würzburg (Kreis)
5,"Untergartelshausen, Freising",48.42810,11.75164,Deutschland,Freising
6,"Potsdam, Brandenburg",52.39407,13.10641,Deutschland,Brandenburg


In [40]:
jobs["created"] = pd.to_datetime(jobs["created"])
jobs["job_age_days"] = (jobs["created"].max() - jobs["created"]).dt.days
jobs["job_age_days"].head()


job_id
5485243050    11
5494004615     5
5489819814     7
5482662909    12
5467966903    21
Name: job_age_days, dtype: int64

In [35]:
jobs["salary_is_predicted"] = jobs["salary_is_predicted"].astype(int)
jobs["salary_is_predicted"].value_counts(dropna=False)


salary_is_predicted
0    50
Name: count, dtype: int64

In [37]:
# One-hot encode selected columns
categorical_cols = ["title", "contract_type", "contract_time", "company_id", "location_id"]
df = pd.get_dummies(jobs, columns=categorical_cols, drop_first=True)
df.head()


,created,adref,redirect_url,salary_is_predicted,category_id,job_age_days,title_(Senior) Softwareentwickler Java / Java EE (m/w/d) ab jetzt,title_.NET Software Engineer,title_Bauingenieur Gebäudemanagement (m/w/d),title_CNC-Dreher (m/w/d),...,location_id_40,location_id_41,location_id_42,location_id_44,location_id_45,location_id_46,location_id_47,location_id_48,location_id_49,location_id_50
job_id,,,,,,,,,,,,,,,,,,,,,
5485243050,2025-11-07 13:05:18,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5485243050?utm_m...,0,1,11,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5494004615,2025-11-13 15:51:45,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ5NDAwNDYxNSIsI...,https://www.adzuna.de/details/5494004615?utm_m...,0,1,5,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5489819814,2025-11-10 21:16:57,eyJhbGciOiJIUzI1NiJ9.eyJpIjoiNTQ4OTgxOTgxNCIsI...,https://www.adzuna.de/details/5489819814?utm_m...,0,1,7,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5482662909,2025-11-05 20:35:43,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5482662909?utm_m...,0,1,12,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5467966903,2025-10-28 14:05:56,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiTXFZR2pmWEU4Qkd5Q...,https://www.adzuna.de/details/5467966903?utm_m...,0,1,21,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
X = df.drop(columns=["adref","redirect_url", "salary_is_predicted"])
y = jobs["salary_is_predicted"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Int64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'numpy.dtypes.BoolDType'>, <class 'nu